# 多GPU训练

In [8]:
%matplotlib inline
import sys
sys.path.append('..')
import torch
from torch import nn
from torch.nn import functional as F
import d2l

In [9]:
# 初始化模型参数
scale = 0.01
W1 = torch.randn(size=(20, 1, 3, 3)) * scale 
b1 = torch.zeros(20)
W2 = torch.randn(size=(50, 20, 5, 5)) * scale 
b2 = torch.zeros(50)
W3 = torch.randn(size=(800, 128)) * scale
b3 = torch.zeros(128)
W4 = torch.randn(size=(128, 10)) * scale
b4 = torch.zeros(10)
params = [W1, b1, W2, b2, W3, b3, W4, b4]
# 定义模型
def lenet(X, params):
     h1_conv = F.conv2d(input=X, weight=params[0], bias=params[1])
     h1_activation = F.relu(h1_conv)
     h1 = F.avg_pool2d(input=h1_activation, kernel_size=(2, 2), stride=(2, 2))
     h2_conv = F.conv2d(input=h1, weight=params[2], bias=params[3])
     h2_activation = F.relu(h2_conv)
     h2 = F.avg_pool2d(input=h2_activation, kernel_size=(2, 2), stride=(2, 2))
     h2 = h2.reshape(h2.shape[0], -1)
     h3_linear = torch.mm(h2, params[4]) + params[5]
     h3 = F.relu(h3_linear)
     y_hat = torch.mm(h3, params[6]) + params[7]
     return y_hat
# 交叉熵损失函数
loss = nn.CrossEntropyLoss(reduction='none')

In [10]:
def get_params(params, device):
    new_params = [p.to(device) for p in params]
    for p in new_params:
        p.requires_grad_()
    return new_params

In [11]:
new_params = get_params(params, d2l.try_gpu(0)) 
print('b1 权重:', new_params[1])
print('b1 梯度:', new_params[1].grad)

b1 权重: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='mps:0', requires_grad=True)
b1 梯度: None


In [12]:
def allreduce(data):
    for i in range(1, len(data)):
        data[0][:] += data[i].to(data[0].device)
    for i in range(1, len(data)):
        data[i][:] = data[0].to(data[i].device)

In [13]:
data = [torch.ones((1, 2), device=d2l.try_gpu(i)) * (i + 1) for i in range(2)] 
print('allreduce之前:\n', data[0], '\n', data[1])
allreduce(data)
print('allreduce之后:\n', data[0], '\n', data[1])

allreduce之前:
 tensor([[1., 1.]], device='mps:0') 
 tensor([[2., 2.]], device='mps:0')
allreduce之后:
 tensor([[3., 3.]], device='mps:0') 
 tensor([[3., 3.]], device='mps:0')


In [14]:
data = torch.arange(20).reshape(4, 5)
devices = [torch.device('mps:0'), torch.device('mps:1')]
split = nn.parallel.scatter(data, devices)
print('input :', data)
print('load into', devices)
print('output:', split)

AttributeError: module 'torch._C' has no attribute '_scatter'